In [10]:
import cma, dill


In [11]:
es = cma.CMAEvolutionStrategy(12 * [0.1], 0.5) 
es.optimize(cma.ff.rosen, iterations=100)
dill.dump(es, open('saved-cma-object.pkl', 'wb'))
print('saved')
del es  # let's start fresh

(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 12 (seed=596132, Thu Aug 22 11:44:09 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     11 1.503587515420888e+02 1.0e+00 4.55e-01  4e-01  5e-01 0:00.0
    2     22 1.476806218326978e+02 1.1e+00 4.04e-01  4e-01  4e-01 0:00.0
    3     33 1.168639227121430e+02 1.2e+00 3.53e-01  3e-01  4e-01 0:00.0
  100   1100 8.042962690762916e+00 6.8e+00 2.75e-02  1e-02  4e-02 0:00.1
saved


In [12]:
es = dill.load(open('saved-cma-object.pkl', 'rb'))
print('resumed')
es.optimize(cma.ff.rosen, verb_disp=200)


resumed
  200   2200 5.244934515194465e+00 1.0e+01 3.17e-02  1e-02  3e-02 0:00.2
  400   4400 4.561792298720520e-01 1.6e+01 2.82e-02  3e-03  2e-02 0:00.4
  600   6600 1.514286904547965e-08 6.0e+01 5.06e-05  2e-06  7e-05 0:00.6
  694   7634 5.692158024555947e-14 6.3e+01 1.66e-07  4e-09  1e-07 0:00.7
  694   7634 5.692158024555947e-14 6.3e+01 1.66e-07  4e-09  1e-07 0:00.7
termination by {'tolfun': 1e-11}
best f-value = 2.4496030103278e-14
solution = [1.         1.         1.         1.         0.99999999 0.99999999
 1.         1.         1.         0.99999999 0.99999998 0.99999996]


In [2]:
import cma, mo, dill

# Defining a callable multiobjective function:

class FitFun:
    def __init__(self, *args):
        self.callables = args
    def __call__(self, x):
        return [f(x) for f in self.callables]
fitness = FitFun(cma.ff.sphere, lambda x: cma.ff.sphere(x-1)) 

# Defining a multiobjective function:

dim = 10
num_kernels = 5

# Instantiating `Sofomore` with `cma-es`, via the `get_cmas` factory function:
#list_of_solvers = mo.get_cmas(num_kernels * [dim * [0.3]], 0.2)

# + Testing the bounds:
inopts = {'bounds':[0.2, None]}
list_of_solvers = mo.get_cmas(num_kernels * [dim * [0.3]], 0.2, inopts)
moes = mo.Sofomore(list_of_solvers, reference_point = [11,11])


In [3]:
#while not moes.stop():
for i in range(100):
    solutions = moes.ask()
    objective_values = [fitness(x) for x in solutions]
    moes.tell(solutions, objective_values)
    moes.disp()
    dill.dump(moes, open('saved-mocma-object.pkl', 'wb'))


Iterat #Fevals   Hypervolume   axis ratios   sigmas   min&max stds
                                  (median)  (median)    (median)
    1     10 6.161000000000001e+01 1.0e+00 2.00e-01  2e-01  2e-01


TypeError: can't pickle generator objects

In [4]:
moes = dill.load(open('saved-mocma-object.pkl', 'rb'))


EOFError: Ran out of input